In [10]:
pip install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [praw]
Note: you may need to restart the kernel to use updated packages.


In [1]:
import praw

reddit = praw.Reddit(
    client_id='zRF0ZieKs1Q2mLgpb76x7g',
    client_secret='uunPz8KIIhhLq0rQp_QkhPejdBC1FQ',
    user_agent='script:sentimientos-app:v1.0 (by u/According-Sherbet796)',
    username='According-Sherbet796',
    password='Adrian123'
)

# Probar que la autenticación fue exitosa
print("Autenticado como:", reddit.user.me())

Autenticado como: According-Sherbet796


In [20]:
pip install textblob

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 624.3/624.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: nltk
    Found existing installation: nltk 3.8.1
    Uninstalling nltk-3.8.1:
      Successfully uninstalled nltk-3.8.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [textblob]
Note: you may need to restart the kernel to use updated packages.


In [3]:
from textblob import TextBlob

In [4]:
# URL de la publicación (puedes cambiarla)
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Cargar todos los comentarios
submission.comments.replace_more(limit=0)  # Cargar todos los comentarios visibles

# Analizar polaridad
for i, comment in enumerate(submission.comments[:20]):  # Puedes ajustar la cantidad de comentarios
    texto = comment.body
    blob = TextBlob(texto)
    polaridad = blob.sentiment.polarity
    print(f"{i+1}. Comentario: {texto[:100]}...")  # muestra solo los primeros 100 caracteres
    print(f"   ➤ Polaridad: {polaridad}\n")

1. Comentario: I cannot imagine having that job at this exact moment......
   ➤ Polaridad: 0.25

2. Comentario: What a horrible day to have a brain...
   ➤ Polaridad: -1.0

3. Comentario: You and me girl...
   ➤ Polaridad: 0.0

4. Comentario: I wanna ask this woman if she had a headache or a downright migraine after witnessing this because I...
   ➤ Polaridad: -0.275

5. Comentario: I had this reaction too. Really she wanted to cry, but she was trying to keep it together....
   ➤ Polaridad: 0.2

6. Comentario: For me, this is much sadder than the argument. She knows the consequences of losing US support. It c...
   ➤ Polaridad: 0.2

7. Comentario: Time to wake up, Europe. History might repeat itself.

edit: I'm European, to be clear, considering ...
   ➤ Polaridad: 0.05000000000000001

8. Comentario: She looks like she's gonna need a lotta Ativan these coming months. I feel for her 😔...
   ➤ Polaridad: 0.0

9. Comentario: Economy crashing, -1.5% GDP growth predicted for Q1, allies aban

In [7]:
# URL del post
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Cargar todos los comentarios visibles
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()

# Crear y guardar el archivo CSV
with open("comentarios_reddit.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(["Comentario", "Polaridad", "Longitud"])

    for comment in comentarios:
        texto = comment.body
        blob = TextBlob(texto)
        polaridad = blob.sentiment.polarity
        writer.writerow([texto, polaridad, len(texto)])

print("✅ Comentarios guardados en comentarios_reddit.csv")

✅ Comentarios guardados en comentarios_reddit.csv


In [8]:
df = pd.read_csv("comentarios_reddit.csv")
df.head()

,Comentario,Polaridad,Longitud
0,I cannot imagine having that job at this exact...,0.250,56
1,What a horrible day to have a brain,-1.000,35
2,You and me girl,0.000,15
3,I wanna ask this woman if she had a headache o...,-0.275,191
4,I had this reaction too. Really she wanted to ...,0.200,90


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Comentario  496 non-null    object 
 1   Polaridad   496 non-null    float64
 2   Longitud    496 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 11.8+ KB


In [13]:
from datetime import datetime

In [25]:
# URL de la publicación
url = "https://www.reddit.com/r/WatchPeopleDieInside/comments/1j0gln4/ukrainian_ambassador_to_us_when_hearing_trump_and/"
submission = reddit.submission(url=url)

# Expandir todos los comentarios
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()  # incluye todos los niveles

datos = []

for comment in comentarios:
    texto = comment.body.strip()
    if not texto:
        continue  # omitir vacíos o eliminados

    autor = comment.author
    if autor:
        nombre = autor.name
        try:
            creado = datetime.utcfromtimestamp(autor.created_utc).strftime('%Y-%m-%d')
        except:
            creado = ""
    else:
        nombre = "[eliminado]"
        creado = ""

    datos.append([nombre, creado, texto])

# Guardar en CSV
with open("comentarios_reddit_con_usuario.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(["Usuario", "Fecha_Creación", "Comentario"])
    writer.writerows(datos)

print(f"✅ Se guardaron {len(datos)} comentarios en 'comentarios_reddit_con_usuario.csv'")

✅ Se guardaron 496 comentarios en 'comentarios_reddit_con_usuario.csv'


In [26]:
df = pd.read_csv("comentarios_reddit_con_usuario.csv")

In [ ]:
# Librerias usadas para la extraccion de comentarios
import praw
from textblob import TextBlob
import csv
import time
from datetime import datetime
import pandas as pd
import os

In [3]:
# URL de la publicación
url = "https://www.reddit.com/r/Hobbies/comments/1frlxq5/what_hobby_unexpectedly_changed_your_life_and_how/"
submission = reddit.submission(url=url)

# Expandir todos los comentarios
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()  # incluye todos los niveles

datos = []

for comment in comentarios:
    texto = comment.body.strip()
    if not texto:
        continue  # omitir vacíos o eliminados

    autor = comment.author
    if autor:
        nombre = autor.name
        try:
            creado = datetime.utcfromtimestamp(autor.created_utc).strftime('%Y-%m-%d')
        except:
            creado = ""
    else:
        nombre = "[eliminado]"
        creado = ""

    datos.append([nombre, creado, texto])

# Guardar en CSV
with open("comentarios_reddit_hobbie_usuarios.csv", mode="w", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    writer.writerow(["Usuario", "Fecha_Creación", "Comentario"])
    writer.writerows(datos)

print(f"✅ Se guardaron {len(datos)} comentarios en 'comentarios_reddit_hobbie_usuarios.csv'")

✅ Se guardaron 493 comentarios en 'comentarios_reddit_hobbie_usuarios.csv'


In [8]:
df = pd.read_csv("comentarios_reddit_hobbie_usuarios.csv")
df

,Usuario,Fecha_Creación,Comentario
0,lazygramma,2023-01-05,"Jigsaw puzzles. For me, when I puzzle I move ..."
1,Vegetable_Size9918,2024-06-21,"Gardening; after my mom committed suicide, I w..."
2,cavviecreature,2020-08-24,"knitting. I didn't even want to knit, I wanted..."
3,[eliminado],NaN,Boxing. I never realized a gym was a stones th...
4,Yrch84,2021-12-28,Tabletop Wargaming.\n\n\nI acutally rediscover...
...,...,...,...
488,mikareno,2021-08-20,I've actually heard of the couch to 5k program...
489,Inside_Indication993,2023-01-09,Right.. like the money aspect isn't even a par...
490,AdministrativeKick42,2020-08-09,I love it. Truly.
491,VinceInMT,2024-05-26,I’ve had a darkroom since getting out of the m...


In [13]:
import os

In [ ]:
# Links de los post de reddit utilizados para los hobbies
"https://www.reddit.com/r/Hobbies/comments/1frlxq5/what_hobby_unexpectedly_changed_your_life_and_how/"
"https://www.reddit.com/r/Hobbies/comments/17x20ht/what_are_some_lesser_known_creative_hobbies/"
"https://www.reddit.com/r/AskReddit/comments/1lcb4s1/whats_the_cheapest_hobby_someone_can_get_into/"
"https://www.reddit.com/r/AskReddit/comments/1lmebln/what_is_the_most_expensive_hobby_you_have/"


In [ ]:
# Links de los post de reddit usados para un evento deportivo a nivel mundial
"https://www.reddit.com/r/formula1/comments/1m8vgxe/2025_belgian_gp_free_practice_1_discussion/"
"https://www.reddit.com/r/formula1/comments/1m9q1s0/max_verstappen_wins_the_sprint_of_the_2025/"
"https://www.reddit.com/r/formula1/comments/1m9ukap/chris_medland_hamilton_has_his_lap_time_deleted/"
"https://www.reddit.com/r/formula1/comments/1m9vpy0/2025_belgian_gp_qualifying_classification/"
"https://www.reddit.com/r/formula1/comments/1m9w0ju/lewis_hamilton_from_my_side_another_mistake_i/"
"https://www.reddit.com/r/formula1/comments/1ma1h9v/kimi_antonelli_emotional_and_in_tears_in_the/"
"https://www.reddit.com/r/formula1/comments/1kv48ks/2025_monaco_gp_post_race_discussion/"
"https://www.reddit.com/r/formula1/comments/1lngbrf/2025_austrian_gp_post_race_discussion/"
"https://www.reddit.com/r/formula1/comments/1lt4am5/2025_british_gp_postrace_discussion/"
"https://www.reddit.com/r/formula1/comments/1kzxkva/2025_spanish_gp_qualifying_discussion/"
"https://www.reddit.com/r/formula1/comments/1kv1vbn/2025_monaco_gp_race_thread/"
"https://www.reddit.com/r/formula1/comments/1lsc4jh/2025_british_gp_postqualifying_discussion/"
"https://www.reddit.com/r/formula1/comments/1jsnrk7/2025_japanese_gp_post_race_discussion/"

In [59]:
# 👉 CAMBIA ESTA URL por la nueva publicación de Reddit
url = "https://www.reddit.com/r/formula1/comments/1jsnrk7/2025_japanese_gp_post_race_discussion/"
submission = reddit.submission(url=url)

# Expandir todos los comentarios
submission.comments.replace_more(limit=0)
comentarios = submission.comments.list()

# Prepara los datos
datos = []

for comment in comentarios:
    texto = comment.body.strip()
    if not texto:
        continue

    autor = comment.author
    if autor:
        nombre = autor.name
        try:
            creado = datetime.utcfromtimestamp(autor.created_utc).strftime('%Y-%m-%d')
        except:
            creado = ""
    else:
        nombre = "[eliminado]"
        creado = ""

    datos.append([nombre, creado, texto])

    # Espera 1 segundos entre cada comentario
    time.sleep(1)

In [60]:
# Guardar en CSV (crear o añadir según exista)
archivo = "comentarios_reddit_deporte_mundial_F1.csv"
archivo_nuevo = not os.path.exists(archivo)

with open(archivo, mode="a", newline="", encoding="utf-8") as archivo_csv:
    writer = csv.writer(archivo_csv)
    if archivo_nuevo:
        writer.writerow(["Usuario", "Fecha_Creación", "Comentario"])
    writer.writerows(datos)

print(f"🏁 Se guardaron {len(datos)} comentarios en '{archivo}'")

🏁 Se guardaron 493 comentarios en 'comentarios_reddit_deporte_mundial_F1.csv'


In [61]:
df = pd.read_csv("comentarios_reddit_deporte_mundial_F1.csv")
df

,Usuario,Fecha_Creación,Comentario
0,yeahmatenomate,2019-06-17,The break between Silverstone and Spa has felt...
1,SpaceOdysseus23,2021-01-08,"Ollie said ""Fuck them kids"""
2,james_d21,2012-11-15,Spa Rancorchamps https://i.imgur.com/JPKpiiv.jpeg
3,elektricniorgazam,2020-06-01,Lol Yuki does somewhat well and Max is immedia...
4,A___99,2021-02-09,"Piastri has done very well here before, looks ..."
...,...,...,...
5366,Zed_or_AFK,2013-05-26,At least he had a few years to adapt to suckin...
5367,Swagtarded,2011-06-16,And lando forcing himself off track!
5368,Poolix,2013-07-02,I mean I get it's Crofty's job to generate exc...
5369,ForeverAddickted,2016-10-29,"""I'd rather drive a Sauber without an engine"" ..."
